In [ ]:
import os
os.chdir("/Users/mphomafata/Documents/GitHub/Chromatographic-data")
import pandas as pd
from scipy.io import netcdf_file
beer_files = sorted(os.listdir("/Users/mphomafata/Documents/Work_file/Collaborative Work/Cody/Untargeted - Gin and Beer/raw_data/Beer/"))
beer_data = []
for beer_file in beer_files[:3]:
    cdf_file = netcdf_file(f'/Users/mphomafata/Documents/Work_file/Collaborative Work/Cody/Untargeted - Gin and Beer/raw_data/Beer/{beer_file}', mmap=False)
    data = pd.DataFrame(columns=["retention_time","mass_values", "intensities"])
    # variables of interest
    mass_values = cdf_file.variables['mass_values'].data
    retention_time = cdf_file.variables['scan_acquisition_time'].data
    point_count = cdf_file.variables['point_count'].data
    
    # cumulative counter 
    def Cumulative(lists): 
        cu_list = [] 
        length = len(lists) 
        cu_list = [sum(lists[0:x:1]) for x in range(0, length+1)] 
        return cu_list[1:]
    lists = point_count
    point_counter = Cumulative(lists)
    
    for i, s in enumerate(retention_time):
        if i==0:
            start = i
        else:
            start = point_counter[i-1]+1
        stop = point_counter[i]
        rt = cdf_file.variables['scan_acquisition_time'][i]
        # print("retention time " + str(rt)+ " has " + str(point_count[i]) + " points starting at " + str(start) + " ending at " + str(stop))
        intensities = cdf_file.variables['intensity_values'][start:stop]
        mass_values = cdf_file.variables['mass_values'][start:stop]
        data.loc[len(data.index)] = [rt,mass_values, intensities]
    data = data.explode(['mass_values','intensities']).reset_index(drop=True)
    # data = data.explode('intensities')
    beer_data.append(data)
    # data.to_excel(f"{beer_file}.xlsx") # too large  to print to excel
    print("Finished sample " + str(beer_files.index(f"{beer_file}") +1) + " of " + str(len(beer_files)))
    # print(f"Sample {beer_file} completed!")
cdf_file .close()


# Peak detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
x = pd.DataFrame(beer_data[0])
x=x.groupby(["retention_time"]).aggregate({'intensities':'sum'}).reset_index()
x=x["intensities"].reset_index(drop=True)
# x=x.iloc[1500:2500].reset_index(drop=True)
peaks, _ = find_peaks(x, width=10, threshold=10)

fig = plt.figure(figsize = (12, 10))
plt.plot(x,'black')
plt.plot(peaks, x[peaks], "x",color = 'red') 


# Plot the 2D and 3D spectrum for the first sample
# 2D Plot


In [ ]:
import matplotlib.pyplot as plt
data = beer_data[0]
x = data["retention_time"]/60
y = data["mass_values"]
z = data["intensities"]

# Creating figure
fig= plt.figure(figsize = (12, 10))
 
# Creating plot
plt.scatter(x, y, 
             c = 'green', 
             marker = "x", s= 2, alpha = 0.01)
plt.scatter(x, y,
            c = 'red', 
             marker = "x", s= 2, alpha = z/(max(z)))
plt.xlabel('Retention time (minutes)', fontweight ='bold') 
plt.ylabel('Mass-to-charge (mz ratio)', fontweight ='bold') 
plt.savefig(fname = "2d_plot.jpg",dpi = 600,
                        transparent=True, format = "jpg")
plt.show()

# 3D Plot

In [ ]:
import matplotlib.pyplot as plt
data = beer_data[0]

# Creating figure
fig = plt.figure(figsize = (12, 10))
ax = fig.add_subplot(projection='3d')

ax.scatter(xs=data["retention_time"]/60,
           ys = data["mass_values"], 
           zs = data["intensities"], 
           c = data["intensities"]
           ,
           marker = "x", s= 1, alpha = 0.5)

ax.set_xlabel('Retention time (minutes)', fontweight ='bold') 
ax.set_ylabel('Mass-to-charge (mz ratio)', fontweight ='bold') 
ax.set_zlabel('Intensity', fontweight ='bold')
plt.savefig(fname = "3d_plot.jpg",dpi = 300,
                        transparent=True, format = "jpg")
plt.show()

# Pivot data wider

In [ ]:
import pandas as pd

dataset1 = pd.DataFrame(beer_data[0]).pivot(index='retention_time', columns='mass_values', values='intensities')
dataset1 = dataset1.fillna(0)
# dataset1.to_csv("dataset1.csv")
dataset2 = pd.DataFrame(beer_data[1]).pivot(index='retention_time', columns='mass_values', values='intensities')
dataset2 = dataset2.fillna(0)
# dataset2.to_csv("dataset2.csv")
dataset3 = pd.DataFrame(beer_data[2]).pivot(index='retention_time', columns='mass_values', values='intensities')
dataset3 = dataset3.fillna(0)
# dataset3.to_csv("dataset3.csv")


# MULTIVARIATE ANALYSIS

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from adjustText import adjust_text

dataset1 = pd.DataFrame(beer_data[0]).pivot(columns='mass_values', values='intensities')
dataset1 = dataset1.fillna(0)
my_data = dataset1

features = my_data.columns
df = my_data

x = df.loc[:, features].values

# Separating out the target
y = df.loc[:,["retention_time"]].values

# Standardizing the features
x = StandardScaler().fit_transform(x)

# get component projections
pca = PCA(n_components= 10)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)
finalDf = pd.concat([principalDf, df[["retention_time"]]], axis = 1)

loadings = pd.DataFrame((pca.components_.T * (pca.explained_variance_)), index=features)

# VISUALISE
plt.figure(figsize = (20,15))
plt.xlabel(('PC1 (' + str(round(((pca.explained_variance_ratio_)[0]*100),2))+"%)" ), fontsize = 12, loc="center")
plt.ylabel(('PC2 ('+ str(round(((pca.explained_variance_ratio_)[1]*100),2))+"%)" ), fontsize = 12, loc="center")
plt.axhline(0, color='black', linestyle = "-.", linewidth = 1)
plt.axvline(0, color='black', linestyle = "-.", linewidth = 1)

# SCORES
labels = finalDf["retention_time"]
plt.scatter(finalDf[0],finalDf[1], c= "green", marker="D", s= 200, alpha=0.5)
scores_labels = []
for i in range(0, len(labels)):
    scores_labels.append(plt.text(finalDf[0].iloc[i],
            finalDf[1].iloc[i],
             f'{labels[i]}',
             fontsize=10,
             horizontalalignment="center",
             verticalalignment='center')
    )
# adjust_text(scores_labels, only_move={'points':'x', 'texts':'x'}, arrowprops=dict(arrowstyle="->", color='grey', lw=0.8))
# LOADINGS
plt.scatter(loadings[0], loadings[1], marker="v", c="red")
loadings_labels = []
for i in range(0, len(features)):
   loadings_labels.append(
   plt.text(loadings[0].iloc[i],
            loadings[1].iloc[i],
             f'{features[i]}',
             fontsize=10,
             horizontalalignment="center",
             verticalalignment='bottom',
             # bbox=dict(facecolor='grey', alpha=0.2),
             # style='italic'
             )
   )
# adjust_text(loadings_labels, only_move={'points':'y', 'texts':'y'}, arrowprops=dict(arrowstyle="->", color='grey', lw=0.8))

# SAVE THE GRAPH
plt.savefig(fname='PCA_concat.jpg', dpi=600, bbox_inches=None,transparent=True, format="jpg")
